In [1]:
from openai import OpenAI

In [2]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
docs = loader.load()

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=1000, chunk_overlap=0)

In [7]:
splits = splitter.split_documents(docs)

In [8]:
len(splits)

16

In [9]:
splits[0]

Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'title': "LLM Powered Autonomous Agents | Lil'Log", 'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, thereby improving the quality of final resul

In [10]:
def summarise_text(text:str) -> str:
    client = OpenAI()
    summary = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "Produce a concise summary of the following text."},
            {"role": "user", "content": text},
        ]
    ).choices[0].message.content
    return summary

In [11]:
summaries = [summarise_text(split.page_content) for split in splits]

In [13]:
summaries

['The article "LLM Powered Autonomous Agents" by Lilian Weng explores the development of autonomous agents using large language models (LLMs) as their central controller. It outlines key system components including planning—where agents decompose tasks and engage in self-reflection; memory—distinguished between short-term and long-term for information retention; and tool use—where agents utilize external APIs for additional resources. The article highlights various proof-of-concept demonstrations like AutoGPT and BabyAGI, emphasizing the potential of LLMs to function as general problem solvers beyond mere content generation.',
 'The text discusses the components of a Large Language Model (LLM)-powered autonomous agent system, focusing on planning and self-reflection. Planning involves task decomposition techniques such as Chain of Thought (CoT) and Tree of Thoughts, which break complex tasks into manageable steps and explore multiple reasoning pathways. An external classical planner (L

In [16]:
def combine_summaries(summaries:list[str]) -> str:
    client = OpenAI()
    combined = "\n".join(summaries)
    summary = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "Produce a final, reduced, and concise summary of the following list of summaries."},
            {"role": "user", "content": combined},
        ]
    ).choices[0].message.content
    return summary

In [17]:
combine_summaries(summaries)

'The article "LLM Powered Autonomous Agents" by Lilian Weng discusses the development of autonomous agents driven by large language models (LLMs), outlining key components such as planning, memory, and tool use. It emphasizes task decomposition techniques like Chain of Thought and introduces methods for self-reflection, including ReAct and Reflexion, to enhance decision-making. Algorithms like Chain of Hindsight and Algorithm Distillation improve model performance, while memory types (sensory, short-term, long-term) and efficient retrieval methods (e.g., MIPS) are explored.\n\nThe article highlights MRKL, a neuro-symbolic architecture, and frameworks like TALM and Toolformer to enhance LLM capabilities through external tools. The API-Bank is introduced to evaluate LLM performance in task execution. Additionally, developments in anticancer drug discovery and simulative Generative Agents showcase complex behaviors. However, limitations of LLMs, such as context length constraints and reli